# Classificador K-means com os dados do ENADE

## Normalização dos dados (Z-score)

### 1. Inicialmente iremos definir as funcoes necessarias para a normalizacao dos dados.

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 15)

- É realizado as seguintes normalizações:
    - Normalização das notas pelo curso
    - Normalização do tempo de formação e tempo ocioso pelo curso
    - Normalização da idade e idade do comeco da graduacao pelo curso

In [2]:
def normalizarValoresNumericos(data, ano):

    if ano == 2008:
        cursos = ['SI', 'BCC', 'ADS','RC', 'EC']

    elif ano == 2011 or ano == 2014:
        cursos = ['SI', 'BCC', 'ADS','RC', 'EC','LCC']

    else:
        cursos = ['SI', 'ADS','BCC', 'RC', 'EC','LCC','GTI']

    columnsToNormalize = [
        'nota_geral', 
        # 'nota_formacao_geral', 
        # 'nota_obj_formacao_geral', 
        # 'nota_dis_formacao_geral', 
        # 'nota_componente_especifico', 
        # 'nota_obj_componente_especifico', 
        # 'nota_dis_componente_especifico', 
        'tempo_cursado', 
        'idade_comeco_graduacao', 
        'tempo_ocioso', 
        'idade'
    ]

    for curso in cursos:

        dataCurso = data.loc[data['nome_curso'] == curso]
        
        scaler = StandardScaler()
        scaler.fit(dataCurso[columnsToNormalize])

        dataCurso[columnsToNormalize] = scaler.transform(dataCurso[columnsToNormalize])
        data.loc[data['nome_curso'] == curso] = dataCurso

    return data

In [3]:
def uf(data):

    scaler = OneHotEncoder()
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['uf']]).toarray())

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = scalerDataFrame.rename(columns={
        0: 'AC',
        1: 'AL',
        2: 'AM',
        3: 'AP',
        4: 'BA',
        5: 'CE',
        6: 'DF',
        7: 'ES',
        8: 'GO',
        9: 'MA',
        10: 'MG',
        11: 'MS',
        12: 'MT',
        13: 'PA',
        14: 'PB',
        15: 'PE',
        16: 'PI',
        17: 'PR',
        18: 'RJ',
        19: 'RN',
        20: 'RO',
        21: 'RR',
        22: 'RS',
        23: 'SC',
        24: 'SE',
        25: 'SP',
        26: 'TO',
    })

    data = data.join(scalerDataFrame)
    data.drop(['uf'], axis=1, inplace=True)

    return data

In [4]:
def nomeCurso(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['nome_curso']]).toarray())

    missingCursos = ['LCC', 'GTI']

    if ano == 2008:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'ADS',
            1: 'BCC',
            2: 'EC',
            3: 'RC',
            4: 'SI'
        })

        scalerDataFrame[missingCursos] = 0
    
    elif ano == 2011 or ano == 2014:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'ADS',
            1: 'BCC',
            2: 'EC',
            3: 'LCC',
            4: 'RC',
            5: 'SI'
        })

        scalerDataFrame[missingCursos[1]] = 0

    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'ADS',
            1: 'BCC',
            2: 'EC',
            3: 'GTI',
            4: 'LCC',
            5: 'RC',
            6: 'SI'
        })

    data = data.join(scalerDataFrame)
    # data.drop(['nome_curso'], axis=1, inplace=True)

    return data

In [5]:
def regiao(data):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['regiao']]).toarray())

    scalerDataFrame = scalerDataFrame.rename(columns={
        0: 'Regiao CO',
        1: 'Regiao N',
        2: 'Regiao NE',
        3: 'Regiao S',
        4: 'Regiao SE'
    })

    data = data.join(scalerDataFrame)
    data.drop(['regiao'], axis=1, inplace=True)

    return data

In [6]:
def categoriaAdmIES(data):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['categoria_adm_IES']]).toarray())

    scalerDataFrame = scalerDataFrame.rename(columns={
        0: 'IES privada',
        1: 'IES publica'
    })

    data = data.join(scalerDataFrame)
    data.drop(['categoria_adm_IES'], axis=1, inplace=True)

    return data

In [7]:
def turno(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['turno']]).toarray())

    if ano == 2008:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Turno diurno',
            1: 'Turno integral',
            2: 'Turno noturno',
            3: 'Não-Informado'
        })

        scalerDataFrame.drop(['Não-Informado'], axis=1, inplace=True)
    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Turno diurno',
            1: 'Turno integral',
            2: 'Turno noturno'
        })

    data = data.join(scalerDataFrame)
    data.drop(['turno'], axis=1, inplace=True)

    return data

In [8]:
def tipoEnsinoMedio(data):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['tipo_ensino_medio']]).toarray())

    scalerDataFrame = scalerDataFrame.rename(columns={
        0: 'Não-Informado',
        1: 'EM particular',
        2: 'EM publico'
    })

    scalerDataFrame.drop(['Não-Informado'], axis=1, inplace=True)

    data = data.join(scalerDataFrame)
    data.drop(['tipo_ensino_medio'], axis=1, inplace=True)

    return data

In [9]:
def raca(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['cor_raca']]).toarray())

    if ano == 2008 or ano == 2017:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Raca amarela',
            1: 'Raca branca',
            2: 'Raca indígena',
            3: 'Não Inf.',
            4: 'Raca parda',
            5: 'Raca preta'
        })

        scalerDataFrame.drop(['Não Inf.'], axis=1, inplace=True)

    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Raca amarela',
            1: 'Raca branca',
            2: 'Raca indígena',
            3: 'Raca parda',
            4: 'Raca preta'
        })
        
    data = data.join(scalerDataFrame)
    data.drop(['cor_raca'], axis=1, inplace=True)

    return data

In [10]:
def escolaridadePaiEMae(data):

    labelEscolaridade = ['escolaridade_pai', 'escolaridade_mae']

    for idx, column in enumerate(labelEscolaridade):

        scaler = OneHotEncoder()

        # os valores são ordenados em ordem alfabética
        scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[[column]]).toarray())

        if idx == 0:
            label = 'Pai '
        else:
            label = 'Mae '

        scalerDataFrame = scalerDataFrame.rename(columns={
            0: label + 'EF',
            1: label + 'EM',
            2: label + 'ES',
            3: 'Nenhuma'
        })

        scalerDataFrame.drop(['Nenhuma'], axis=1, inplace=True)

        data = data.join(scalerDataFrame)
        data.drop([column], axis=1, inplace=True)

    return data

In [11]:
def estadoCivil(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['estado_civil']]).toarray())
    
    if ano == 2011:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Casado',
            1: 'Divorciado',
            2: 'Não-Informado',
            3: 'Outro',
            4: 'Solteiro',
            5: 'Viúvo'
        })
    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Casado',
            1: 'Divorciado',
            2: 'Outro',
            3: 'Solteiro',
            4: 'Viúvo'
        })

    scalerDataFrame.drop(['Outro'], axis=1, inplace=True)

    if ano == 2011:
        scalerDataFrame.drop(['Não-Informado'], axis=1, inplace=True)

    data = data.join(scalerDataFrame)
    data.drop(['estado_civil'], axis=1, inplace=True)

    return data

In [12]:
def moradia(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['moradia']]).toarray())

    if ano == 2008:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Moradia cônjuge/filhos',
            1: 'Moradia individual/coletiva',
            4: 'Moradia parentes',
            5: 'Moradia universidade'
        })
        
        scalerDataFrame.drop([2], axis=1, inplace=True)
        scalerDataFrame.drop([3], axis=1, inplace=True)
        scalerDataFrame['Moradia sozinho'] = 0
        scalerDataFrame['Moradia outro'] = 0

    elif ano == 2011:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Moradia cônjuge/filhos',
            1: 'Moradia individual/coletiva',
            2: 'Não-Informado',
            3: 'Moradia outro',
            4: 'Moradia parentes',
            5: 'Moradia sozinho',
            6: 'Moradia universidade'
        })

        scalerDataFrame.drop(['Não-Informado'], axis=1, inplace=True)

    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'Moradia cônjuge/filhos',
            1: 'Moradia individual/coletiva',
            2: 'Moradia outro',
            3: 'Moradia parentes',
            4: 'Moradia sozinho',
            5: 'Moradia universidade'
        })
        
    scalerDataFrame.drop(['Moradia outro'], axis=1, inplace=True)

    data = data.join(scalerDataFrame)
    data.drop(['moradia'], axis=1, inplace=True)

    return data

In [13]:
def ensinoMedio(data, ano):

    scaler = OneHotEncoder()

    # os valores são ordenados em ordem alfabética
    scalerDataFrame = pd.DataFrame(scaler.fit_transform(data[['ensino_medio']]).toarray())

    if ano == 2011:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'EM EJA/Supletivo',
            1: 'EM tradicional',
            3: 'Outro',
            4: 'EM magistério',
            5: 'EM técnico'
        })

        scalerDataFrame.drop([2], axis=1, inplace=True)

    else:
        scalerDataFrame = scalerDataFrame.rename(columns={
            0: 'EM EJA/Supletivo',
            1: 'EM tradicional',
            2: 'Outro',
            3: 'EM magistério',
            4: 'EM técnico'
        })

    scalerDataFrame.drop(['Outro'], axis=1, inplace=True)

    data = data.join(scalerDataFrame)
    data.drop(['ensino_medio'], axis=1, inplace=True)

    return data

In [14]:
def transformarValoresCategoricos(data, ano):

    data = uf(data)
    data = nomeCurso(data, int(ano))
    data = regiao(data)
    data = categoriaAdmIES(data)
    data = turno(data, int(ano))
    data = tipoEnsinoMedio(data)
    data = raca(data, int(ano))
    data = escolaridadePaiEMae(data)
    data = estadoCivil(data, int(ano))
    data = moradia(data, int(ano))
    data = ensinoMedio(data, int(ano))

    return data

### 2. Abertura dos arquivos, e tratamento dos dados pelas funções definidas acima 

In [15]:
anos = ['2008', '2011', '2014', '2017']

for ano in anos:
    data = pd.read_csv('data/not_norm/dados'+ ano + '_nn.csv', sep=',')

    # tirar notas zeradas
    data.drop(data.loc[data['nota_geral'] < 1].index.tolist())

    # transformar valores categoricos
    data = transformarValoresCategoricos(data, ano)
    data.to_csv('data/not_norm/dados' + ano + '_nn.csv', index=False)

    # normalizacao valores numericos
    data = normalizarValoresNumericos(data, int(ano))
    data.drop(['nome_curso'], axis=1, inplace=True)

    data.to_csv('data/norm/dados' + ano + '_n.csv', index=False)

C:\ferramentass\Python3.9\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\ferramentass\Python3.9\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\ferramentass\Python3.9\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

### 3. Após todo o processo, junto novamente todos os dados em um unico arquivo .csv

In [16]:
arr = ['n', 'nn']
dir = ['norm/', 'not_norm/']

for idx, label in enumerate(arr):
    data2008 = pd.read_csv('data/'+ dir[idx] +'dados2008_' + label + '.csv', sep=',')
    data2011 = pd.read_csv('data/'+ dir[idx] +'dados2011_' + label + '.csv', sep=',')
    data2014 = pd.read_csv('data/'+ dir[idx] +'dados2014_' + label + '.csv', sep=',')
    data2017 = pd.read_csv('data/'+ dir[idx] +'dados2017_' + label + '.csv', sep=',')

    frames = [data2008, data2011, data2014, data2017]

    data = pd.concat(frames)

    data.to_csv('data/dados_enade_' + label + '.csv', index=False)